<a href="https://colab.research.google.com/github/SandroMuradashvili/MTL_student_performance/blob/main/MTL_student_performance.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1124]:
import pandas as pd
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.compose import ColumnTransformer
from sklearn.model_selection import train_test_split
import torch
from torch.utils.data import Dataset, DataLoader
import torch
import torch.nn as nn
import torch
import torch.nn as nn
from torch.optim import Adam
from sklearn.metrics import mean_absolute_error, accuracy_score, f1_score
import torch


In [1125]:
# -----------------------------
# 1. Load the data | STEP 1
# -----------------------------
url = "https://raw.githubusercontent.com/SandroMuradashvili/MTL_student_performance/refs/heads/main/data/student-por.csv"

# Read CSV with correct separator
#df = pd.read_csv(url, sep=';')
df = pd.read_csv(url, sep=';', header=0)  # explicitly use first row as header


# Strip any leading/trailing whitespace from column names
df.columns = df.columns.str.strip()


# Check columns
print(df.columns.tolist())

['school', 'sex', 'age', 'address', 'famsize', 'Pstatus', 'Medu', 'Fedu', 'Mjob', 'Fjob', 'reason', 'guardian', 'traveltime', 'studytime', 'failures', 'schoolsup', 'famsup', 'paid', 'activities', 'nursery', 'higher', 'internet', 'romantic', 'famrel', 'freetime', 'goout', 'Dalc', 'Walc', 'health', 'absences', 'G1', 'G2', 'G3']


In [1126]:
#TEST
# Show first 5 rows to understand the data
print("First 5 rows of raw dataset:")
print(df.head())
# Check columns
print(df.columns.tolist())

First 5 rows of raw dataset:
  school sex  age address famsize Pstatus  Medu  Fedu     Mjob      Fjob  ...  \
0     GP   F   18       U     GT3       A     4     4  at_home   teacher  ...   
1     GP   F   17       U     GT3       T     1     1  at_home     other  ...   
2     GP   F   15       U     LE3       T     1     1  at_home     other  ...   
3     GP   F   15       U     GT3       T     4     2   health  services  ...   
4     GP   F   16       U     GT3       T     3     3    other     other  ...   

  famrel freetime  goout  Dalc  Walc health absences  G1  G2  G3  
0      4        3      4     1     1      3        4   0  11  11  
1      5        3      3     1     1      3        2   9  11  11  
2      4        3      2     2     3      3        6  12  13  12  
3      3        2      2     1     1      5        0  14  14  14  
4      4        3      2     1     2      5        0  11  13  13  

[5 rows x 33 columns]
['school', 'sex', 'age', 'address', 'famsize', 'Pstatus', '

In [1127]:
# -----------------------------
# 2. Separate features and targets
# -----------------------------
# X = all columns except targets
# y_grade = final grade (G3)
# y_romantic = binary (0/1) for romantic relationship
X = df.drop(columns=['G3', 'romantic'])
y_grade = df['G3']
y_romantic = df['romantic'].map({'yes': 1, 'no': 0})  # convert to 0/1


In [1128]:
# -----------------------------
# 3. Split data into train, val, test
# -----------------------------
# First split: train+val vs test (12% test)
X_temp, X_test, y_grade_temp, y_grade_test, y_romantic_temp, y_romantic_test = train_test_split(
    X, y_grade, y_romantic, test_size=0.12, random_state=42, shuffle=True
)

# Second split: train vs val (15% of train+val becomes validation)
X_train, X_val, y_grade_train, y_grade_val, y_romantic_train, y_romantic_val = train_test_split(
    X_temp, y_grade_temp, y_romantic_temp, test_size=0.15, random_state=42, shuffle=True
)

#ADJUST 1 | 0.12 & 0.15


In [1129]:
#Test
# Print dataset sizes
print("\nDataset sizes after splitting:")
print(f"Train: {X_train.shape[0]} samples (~75%)")
print(f"Validation: {X_val.shape[0]} samples (~13%)")
print(f"Test: {X_test.shape[0]} samples (~12%)")


Dataset sizes after splitting:
Train: 485 samples (~75%)
Validation: 86 samples (~13%)
Test: 78 samples (~12%)


In [1130]:
# -----------------------------
# 4. Identify categorical and numerical columns
# -----------------------------
categorical_cols = X_train.select_dtypes(include=['object']).columns.tolist()
numerical_cols = X_train.select_dtypes(include=['int64', 'float64']).columns.tolist()

In [1131]:
#Test
print("\nCategorical columns:", categorical_cols)
print("Numerical columns:", numerical_cols)


Categorical columns: ['school', 'sex', 'address', 'famsize', 'Pstatus', 'Mjob', 'Fjob', 'reason', 'guardian', 'schoolsup', 'famsup', 'paid', 'activities', 'nursery', 'higher', 'internet']
Numerical columns: ['age', 'Medu', 'Fedu', 'traveltime', 'studytime', 'failures', 'famrel', 'freetime', 'goout', 'Dalc', 'Walc', 'health', 'absences', 'G1', 'G2']


In [1132]:
# -----------------------------
# 5. Preprocessing: One-hot encode categorical, standardize numerical
# -----------------------------
categorical_transformer = OneHotEncoder(drop='first', sparse_output=False)  # avoid dummy variable trap
numerical_transformer = StandardScaler()

# Combine preprocessing steps
preprocessor = ColumnTransformer(
    transformers=[
        ('num', numerical_transformer, numerical_cols),
        ('cat', categorical_transformer, categorical_cols)
    ]
)

# Fit the preprocessor only on training data to avoid data leakage
preprocessor.fit(X_train)

# Transform train, val, test sets
X_train_proc = preprocessor.transform(X_train)
X_val_proc = preprocessor.transform(X_val)
X_test_proc = preprocessor.transform(X_test)

In [1133]:
#Test
# Debug prints: check shapes after preprocessing
# print("\nPreprocessing done. Shapes of processed datasets:")
# print("X_train_proc:", X_train_proc.shape)
# print("X_val_proc:", X_val_proc.shape)
# print("X_test_proc:", X_test_proc.shape)

print("X_train_proc:", X_train_proc)


X_train_proc: [[ 0.16583729  1.34192829  1.58561977 ...  0.          1.
   1.        ]
 [ 0.97827249 -1.32179021 -1.17074434 ...  1.          0.
   1.        ]
 [-0.64659791  1.34192829  1.58561977 ...  1.          0.
   1.        ]
 ...
 [-0.64659791 -0.43388404 -0.25195631 ...  1.          1.
   1.        ]
 [ 0.16583729  0.45402212 -0.25195631 ...  1.          1.
   0.        ]
 [ 0.97827249  0.45402212  0.66683173 ...  1.          1.
   1.        ]]


In [1134]:

# -----------------------------
# 6. Create PyTorch Dataset class
# -----------------------------
class StudentDataset(Dataset):
    def __init__(self, X, y_grade, y_romantic):
        # Convert numpy arrays / pandas Series to PyTorch tensors
        self.X = torch.tensor(X, dtype=torch.float32)
        self.y_grade = torch.tensor(y_grade.values, dtype=torch.float32).unsqueeze(1)  # regression target
        self.y_romantic = torch.tensor(y_romantic.values, dtype=torch.long)  # classification target

    def __len__(self):
        return len(self.X)

    def __getitem__(self, idx):
        return self.X[idx], self.y_grade[idx], self.y_romantic[idx]

# -----------------------------
# 7. Create datasets and loaders
# -----------------------------
train_dataset = StudentDataset(X_train_proc, y_grade_train, y_romantic_train)
val_dataset = StudentDataset(X_val_proc, y_grade_val, y_romantic_val)
test_dataset = StudentDataset(X_test_proc, y_grade_test, y_romantic_test)

train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=32)
test_loader = DataLoader(test_dataset, batch_size=32)

#ADJUST 2 | batch size=32
# Batch smaller | Often needs smaller LR |
# Batch larger | Can use bigger LR |

print("\nPyTorch DataLoaders ready!")
print("Example batch from train_loader:")
example_batch = next(iter(train_loader))
print("X batch shape:", example_batch[0].shape)
print("y_grade batch shape:", example_batch[1].shape)
print("y_romantic batch shape:", example_batch[2].shape)

print()

#STEP 1 FINISHED


PyTorch DataLoaders ready!
Example batch from train_loader:
X batch shape: torch.Size([32, 40])
y_grade batch shape: torch.Size([32, 1])
y_romantic batch shape: torch.Size([32])



In [1135]:
encoded = pd.get_dummies(df, drop_first=False)
print(encoded.columns)
print("Total features:", len(encoded.columns))


Index(['age', 'Medu', 'Fedu', 'traveltime', 'studytime', 'failures', 'famrel',
       'freetime', 'goout', 'Dalc', 'Walc', 'health', 'absences', 'G1', 'G2',
       'G3', 'school_GP', 'school_MS', 'sex_F', 'sex_M', 'address_R',
       'address_U', 'famsize_GT3', 'famsize_LE3', 'Pstatus_A', 'Pstatus_T',
       'Mjob_at_home', 'Mjob_health', 'Mjob_other', 'Mjob_services',
       'Mjob_teacher', 'Fjob_at_home', 'Fjob_health', 'Fjob_other',
       'Fjob_services', 'Fjob_teacher', 'reason_course', 'reason_home',
       'reason_other', 'reason_reputation', 'guardian_father',
       'guardian_mother', 'guardian_other', 'schoolsup_no', 'schoolsup_yes',
       'famsup_no', 'famsup_yes', 'paid_no', 'paid_yes', 'activities_no',
       'activities_yes', 'nursery_no', 'nursery_yes', 'higher_no',
       'higher_yes', 'internet_no', 'internet_yes', 'romantic_no',
       'romantic_yes'],
      dtype='object')
Total features: 59


In [1136]:
# 1) All columns after one-hot encoding (or just the DataFrame you used for preprocessing)
print("All columns in encoded DataFrame:")
print(encoded.columns.tolist())

# 2) Columns used for training (X)
print("\nColumns used in X (inputs to the model):")
print(X.columns.tolist())

# 3) Number of features in input to the model
print("\nNumber of input features:", X_train_proc.shape[1])

encoded = pd.get_dummies(df, drop_first=False)
print(encoded.columns)
print("Total features:", len(encoded.columns))



All columns in encoded DataFrame:
['age', 'Medu', 'Fedu', 'traveltime', 'studytime', 'failures', 'famrel', 'freetime', 'goout', 'Dalc', 'Walc', 'health', 'absences', 'G1', 'G2', 'G3', 'school_GP', 'school_MS', 'sex_F', 'sex_M', 'address_R', 'address_U', 'famsize_GT3', 'famsize_LE3', 'Pstatus_A', 'Pstatus_T', 'Mjob_at_home', 'Mjob_health', 'Mjob_other', 'Mjob_services', 'Mjob_teacher', 'Fjob_at_home', 'Fjob_health', 'Fjob_other', 'Fjob_services', 'Fjob_teacher', 'reason_course', 'reason_home', 'reason_other', 'reason_reputation', 'guardian_father', 'guardian_mother', 'guardian_other', 'schoolsup_no', 'schoolsup_yes', 'famsup_no', 'famsup_yes', 'paid_no', 'paid_yes', 'activities_no', 'activities_yes', 'nursery_no', 'nursery_yes', 'higher_no', 'higher_yes', 'internet_no', 'internet_yes', 'romantic_no', 'romantic_yes']

Columns used in X (inputs to the model):
['school', 'sex', 'age', 'address', 'famsize', 'Pstatus', 'Medu', 'Fedu', 'Mjob', 'Fjob', 'reason', 'guardian', 'traveltime', 'stud

In [1137]:
class MultiTaskModel(nn.Module):
    def __init__(self, input_dim=40, shared_dim1=128, shared_dim2=64, head_dim=32, dropout=0.3):
        super(MultiTaskModel, self).__init__()

        # -------------------------
        # 1) Shared Body (feature extractor) | STEP 2
        # -------------------------
        self.shared_body = nn.Sequential(
            nn.Linear(input_dim, shared_dim1),   # 40 -> 128
            nn.BatchNorm1d(shared_dim1),
            nn.ReLU(),
            nn.Dropout(dropout),

            nn.Linear(shared_dim1, shared_dim2), # 128 -> 64
            nn.BatchNorm1d(shared_dim2),
            nn.ReLU(),
            nn.Dropout(dropout)
        )

        # -------------------------
        # 2) Head 1 → Grade Regression (predict G3)
        # -------------------------
        self.head_grade = nn.Sequential(
            nn.Linear(shared_dim2, head_dim),    # 64 -> 32
            nn.ReLU(),
            nn.Linear(head_dim, 1)               # Output single number
        )

        # -------------------------
        # 3) Head 2 → Romantic Classification (0/1)
        # -------------------------
        self.head_romantic = nn.Sequential(
            nn.Linear(shared_dim2, head_dim),    # 64 -> 32
            nn.ReLU(),
            nn.Linear(head_dim, 2)               # 2 output logits
        )

    def forward(self, x):
        shared = self.shared_body(x)         # common features
        grade_pred = self.head_grade(shared) # regression output
        romantic_logits = self.head_romantic(shared) # classification logits
        return grade_pred, romantic_logits


In [1138]:

# Define Model

model = MultiTaskModel(input_dim=40)

# -------------------------------
# 1) Loss functions | STEP 3
# -------------------------------
loss_regression = nn.MSELoss()         # for grade prediction (G3)
loss_classification = nn.CrossEntropyLoss()  # for romantic yes/no

# -------------------------------
# 2) Optimizer
# -------------------------------
optimizer = Adam(model.parameters(), lr=0.001)

# -------------------------------
# 3) Training Loop
# -------------------------------
def train_model(model, train_loader, val_loader, alpha=0.5, epochs=30, device="cpu"):
    model.to(device)

    for epoch in range(epochs):
        model.train()
        total_train_loss = 0
        total_grade_loss = 0
        total_rom_loss = 0

        for X, y_grade, y_romantic in train_loader:

            X, y_grade = X.to(device), y_grade.to(device).float()
            y_romantic = y_romantic.to(device).long()

            # Forward pass → two outputs
            pred_grade, romantic_logits = model(X)

            # --- Compute losses separately ---
            grade_loss = loss_regression(pred_grade, y_grade)
            romantic_loss = loss_classification(romantic_logits, y_romantic)

            # --- Combine losses ---
            #total_loss = grade_loss + romantic_loss # original.

            # -----------------------------
            # Weighted loss # BONUS
            # -----------------------------
            # alpha = 0.005   # <--- importance of G3 in % | UPDATE, instead of manual alpha is passed as parameter
            total_loss = alpha * grade_loss + (1 - alpha) * romantic_loss

            # Backpropagation
            optimizer.zero_grad()
            total_loss.backward()
            optimizer.step()

            # accumulate stats for reporting later
            total_train_loss += total_loss.item()
            total_grade_loss += grade_loss.item()
            total_rom_loss   += romantic_loss.item()


        # -----------------------------
        # Validation (no backprop)
        # -----------------------------
        model.eval()
        val_grade_loss = 0
        val_rom_loss = 0

        with torch.no_grad():
            for X, y_grade, y_romantic in val_loader:
                X, y_grade = X.to(device), y_grade.to(device).float()
                y_romantic = y_romantic.to(device).long()

                pred_g, logit_r = model(X)

                val_grade_loss += loss_regression(pred_g, y_grade).item()
                val_rom_loss   += loss_classification(logit_r, y_romantic).item()

        # Print each epoch for debugging purposes
        # print(f"Epoch {epoch+1}/{epochs} | "
        #       f"Train total: {total_train_loss:.4f} | "
        #       f"Grade: {total_grade_loss:.4f} | "
        #       f"Romantic: {total_rom_loss:.4f} || "
        #       f"Val Grade: {val_grade_loss:.4f} | Val Romantic: {val_rom_loss:.4f}")



In [1139]:
# -------------------------------
#  STEP 4
# -------------------------------

def evaluate_on_test(model, test_loader, device="cpu"):
    model.eval()

    true_grade = []
    pred_grade = []

    true_rom = []
    pred_rom = []

    with torch.no_grad():
        for X, y_grade, y_romantic in test_loader:
            X = X.to(device)

            # forward pass
            grade_out, romantic_logits = model(X)

            # ---- Regression ----
            true_grade.extend(y_grade.numpy())
            pred_grade.extend(grade_out.cpu().numpy())

            # ---- Classification ----
            predicted = torch.argmax(romantic_logits, dim=1).cpu()

            true_rom.extend(y_romantic.numpy())
            pred_rom.extend(predicted.numpy())

    # ---- Compute Metrics ----
    mae = mean_absolute_error(true_grade, pred_grade)
    accuracy = accuracy_score(true_rom, pred_rom)
    f1 = f1_score(true_rom, pred_rom, pos_label=1)   # only for "yes" class

    print("\n========== FINAL TEST RESULTS ==========")
    print(f"Mean Absolute Error (G3 Regression): {mae:.4f}")
    print(f"Accuracy (Romantic Status)        : {accuracy:.4f}")
    print(f"F1-Score (Romantic = YES)         : {f1:.4f}")
    print("=========================================")

    return mae, accuracy, f1




In [1142]:
train_model(model, train_loader, val_loader,alpha=0.01, epochs=30)

evaluate_on_test(model, test_loader)

torch.save(model.state_dict(), "best_model.pth")




========== FINAL TEST RESULTS ==========
Mean Absolute Error (G3 Regression): 1.7775
Accuracy (Romantic Status)        : 0.5256
F1-Score (Romantic = YES)         : 0.2745


In [1147]:
model_2 = MultiTaskModel(input_dim=40)

optimizer = Adam(model_2.parameters(), lr=0.001)

train_model(model_2, train_loader, val_loader,alpha=0.5, epochs=30)

evaluate_on_test(model_2, test_loader)

torch.save(model_2.state_dict(), "best_model_2.pth")

# ALPHA




========== FINAL TEST RESULTS ==========
Mean Absolute Error (G3 Regression): 1.9529
Accuracy (Romantic Status)        : 0.6282
F1-Score (Romantic = YES)         : 0.1212


In [1150]:
model_3 = MultiTaskModel(input_dim=40)

optimizer = Adam(model_3.parameters(), lr=0.001)

train_model(model_3, train_loader, val_loader,alpha=0.8, epochs=30)

evaluate_on_test(model_3, test_loader)

torch.save(model_3.state_dict(), "best_model_3.pth")


========== FINAL TEST RESULTS ==========
Mean Absolute Error (G3 Regression): 2.0933
Accuracy (Romantic Status)        : 0.6026
F1-Score (Romantic = YES)         : 0.1143
